<a href="https://colab.research.google.com/github/sincereQK/QK-ML-Data-study/blob/main/stock_LSTM_20250908.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install tensorflow scikit-learn pandas numpy matplotlib pandas-ta

In [ ]:
pip install --upgrade pandas-ta

In [ ]:
# !pip install --upgrade pandas-ta

import pandas as pd
import numpy as np
import pandas_ta as ta
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Dropout, Input
import os
import glob
from datetime import datetime, timedelta
import sys

# --- 1. CSV 파일 동적 검색 및 불러오기 ---

# 사용할 변수들
folder_path = 'CSV'
company_name = '경방'
today_str = datetime.now().strftime('%Y%m%d')
found_file = None

# 파일 검색
try:
    # CSV 폴더가 없으면 에러 발생
    if not os.path.isdir(folder_path):
        raise FileNotFoundError

    for filename in os.listdir(folder_path):
        # 파일명에 오늘 날짜가 포함되고, 지정된 회사 이름으로 끝나는지 확인
        if today_str in filename and filename.endswith(f'{company_name}.csv'):
            found_file = os.path.join(folder_path, filename)
            print(f"파일을 찾았습니다: {found_file}")
            break # 파일을 찾으면 반복 중단

except FileNotFoundError:
    print(f"오류: '{folder_path}' 폴더를 찾을 수 없습니다. 프로그램을 종료합니다.")
    sys.exit()

# 파일을 찾지 못했으면 프로그램 종료
if found_file is None:
    print(f"오류: '{folder_path}' 폴더에서 오늘 날짜({today_str})와 '{company_name}' 이름이 포함된 CSV 파일을 찾을 수 없습니다.")
    sys.exit()

# 찾은 파일을 데이터프레임으로 불러오기
df = pd.read_csv(found_file)


# --- 2. 데이터 전처리 및 기술적 지표 계산 ---
df['일자'] = pd.to_datetime(df['일자'], format='%Y%m%d')
df = df.sort_values('일자').reset_index(drop=True)
df['sma20'] = ta.sma(df['현재가'], length=20)
df['rsi14'] = ta.rsi(df['현재가'], length=14)
df.dropna(subset=['sma20', 'rsi14'], inplace=True)
df = df.reset_index(drop=True)

features = ['시가', '고가', '저가', '현재가', '거래량', 'sma20', 'rsi14']
data = df[features].copy()
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# --- 3. 모델 파일 관리 및 학습/불러오기 로직 ---
# company_name 변수는 이미 위에서 정의되었으므로 그대로 사용
today_str = datetime.now().strftime('%Y%m%d')
model_filename = f'model_{company_name}_{today_str}.keras'
should_train_new_model = True

existing_models = glob.glob(f'model_{company_name}_*.keras')

if existing_models:
    latest_model_file = max(existing_models, key=os.path.getctime)
    print(f"가장 최신 모델 파일: '{latest_model_file}'")

    try:
        model_date_str = latest_model_file.split('_')[-1].replace('.keras', '')
        model_date = datetime.strptime(model_date_str, '%Y%m%d')

        if datetime.now() - model_date < timedelta(days=7):
            print("최신 모델이 7일 이내에 학습되었습니다. 기존 모델을 불러옵니다.")
            model = load_model(latest_model_file)
            should_train_new_model = False
        else:
            print("최신 모델이 7일 이상 경과되었습니다. 새로운 학습이 필요합니다.")
    except (ValueError, IndexError):
        print("모델 파일명에서 날짜를 추출할 수 없습니다. 새로운 학습을 진행합니다.")

if should_train_new_model:
    if not existing_models:
        print("저장된 모델이 없습니다. 새로 모델을 학습하고 저장합니다.")

    train_data_len = int(np.ceil(len(scaled_data) * 0.8))
    train_data = scaled_data[0:train_data_len, :]
    x_train, y_train = [], []
    look_back = 60

    for i in range(look_back, len(train_data)):
        x_train.append(train_data[i-look_back:i, :])
        y_train.append(train_data[i, 3])

    x_train, y_train = np.array(x_train), np.array(y_train)

    model = Sequential([
        Input(shape=(x_train.shape[1], x_train.shape[2])),
        LSTM(units=50, return_sequences=True),
        Dropout(0.2),
        LSTM(units=50, return_sequences=False),
        Dropout(0.2),
        Dense(units=25),
        Dense(units=1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(x_train, y_train, batch_size=32, epochs=50, verbose=1)

    print(f"새로 학습된 모델을 '{model_filename}' 파일로 저장합니다.")
    model.save(model_filename)

# --- 4. 다음 날 주가 예측 ---
look_back = 60
last_look_back_days_scaled = scaled_data[-look_back:]
x_predict = np.reshape(last_look_back_days_scaled, (1, look_back, len(features)))
predicted_price_scaled = model.predict(x_predict)

dummy_predict = np.zeros((1, len(features)))
dummy_predict[:, 3] = predicted_price_scaled
predicted_price = scaler.inverse_transform(dummy_predict)[:, 3][0]

# --- 5. 매매 신호 판단 ---
latest_data = df.iloc[-1]
current_price = latest_data['현재가']
current_sma20 = latest_data['sma20']
current_rsi14 = latest_data['rsi14']

print("\n--- 매매 신호 판단 ---")
print(f"현재 가격: {current_price:,.0f}원")
print(f"다음 날 예측 가격: {predicted_price:,.0f}원")
print(f"현재 20일 이동평균선: {current_sma20:,.0f}원")
print(f"현재 RSI14: {current_rsi14:.2f}")
print("--------------------")

def get_trading_signal(predicted, current, sma20, rsi14):
    is_buy_signal = (predicted > current * 1.02 and current > sma20 and rsi14 < 70)
    is_sell_signal = (predicted < current * 0.98)

    if is_buy_signal:
        print("판단: 📝 매수 신호")
        return 1
    elif is_sell_signal:
        print("판단: 📉 매도 신호")
        return 0
    else:
        print("판단: ☕ 관망 (매매 신호 없음)")
        return None

signal = get_trading_signal(predicted_price, current_price, current_sma20, current_rsi14)
print(f"\n최종 출력: {signal}")